# Verschiedene Ansätze für den Umgang mit Lücken
Dieser Block wurde erstellt, als noch nicht klar war, welche Methoden des maschinellen Lernens in der vorliegenden Arbeit verwendet werden sollten. Aus diesem Grund wurden präventiv verschiedene Verfahren zum Umgang mit fehlenden Werten (Lücken) in den Niederschlagszeitreihen implementiert.

### Interpolation kleiner Lücken und belassen von NaNs in großen Lücken
1. In der Funktion "interpolate_small_gaps" wird die Zeitreihe zunächst auf stündliche Frequenz gebracht ("asfreq("h")"). Fehlende Werte in "Wert in mm" werden zeitbasiert interpoliert ("method="time"") für maximal 6 aufeinanderfolgende Stundenwerte ("limit=6") innerhalb der Zeitreihe. Interpolierte Werte werden dabei in der Spalte "Interpoliert" markiert. Schließlich werden die durch die stündliche Frequenz entstandenen NaNs in Spalten mit konstanten Werten (z.B. "Stations ID") mit dem jeweiligen ersten Eintrag gefüllt.
2. Die Funktion "interpolate_and_big_gaps" wendet die in 1. definierte Interpolation an. Das daraus resultierende DataFrame wird als CSV-Datei gespeichert, wobei die bisherige Endung "_filtered_standardised.csv" durch "_interpolated_and_nan.csv" ersetzt wird.

### Interpolation kleiner Lücken und Maskierung großer Lücken
1. Die Funktion "mask_big_gaps(interpolated)" nutzt das Ergebnis der Funktion "interpolate_small_gaps" und ersetzt alle restlichen Lücken in "Wert in mm" durch den Fehlerwert -999.0. Diese maskierten Werte werden dabei in der Spalte "Maskiert" markiert. Das daraus resultierende DataFrame wird als CSV-Datei gespeichert, wobei die Endung "_filtered_standardised.csv" durch "_interpolated_and_masked.csv" ersetzt wird.
2. Die Funktion "interpolate_and_mask()" kombiniert beide Funktionen.

### Belassen aller NaNs
Die Funktion "keep_nan()" füllt die aus "asfreq("h")" resultierenden NaNs in konstanten Metadatenspalten auf. Die fehlenden Werte innerhalb der Spalte "Wert in mm" werden nicht interpoliert und bleiben NaNs. Das daraus resultierende DataFrame wird als CSV-Datei gespeichert, wobei die Endung "_standardised.csv" durch "_just_nan.csv" ersetzt wird.

### Maskieren aller NaNs
Die Funktion "mask_nans()" ersetzt alle NaNs in "Wert in mm" durch -999.0 und markiert diese in der Spalte "Maskiert". Das daraus resultierende DataFrame wird als CSV-Datei gespeichert, wobei die Endung "_standardised.csv" durch "_just_masked.csv" ersetzt wurde. 

In der vorliegenden Arbeit wurde letztlich nur die NaN-Variante ("..._just_nan.csv") verwendet, da XGBoost und Random Forest mit fehlenden Werten umgehen können.


In [4]:
from pathlib import Path
import pandas as pd
import glob


files=glob.glob("Datensätze/DWD/Filtered-Enhanced-Ba-Wü-Messnetze und Parameternetze-stündlich/*/*_standardised.csv")
print(f"Found {len(files)} files")

# interpolate every gap in the respective csv file
def interpolate_small_gaps():
    interpolated = []
    for p in files:
        p = Path(p)

        # read the respective file as a data frame.
        df = pd.read_csv(p, sep=",", encoding="utf-8")

        # store the data frame as a copy of the data frame to work with it without missunderstandings.
        df = df.copy()
        # define the column "Datum / Uhrzeit" as a real date time.
        df["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

        # convert the column "Wert in mm" to a real numeric value.
        df["Wert in mm"] = df["Wert in mm"].astype(float)

        # maybe "Datum / Uhrzeit" is by any case not sorted.
        df = df.sort_values("Datum / Uhrzeit")
        
        # set the column "Datum / Uhrzeit" as index and in hourly frequence. Now the gaps are rows with NaNs but the Index is a real date time.
        df = df.set_index("Datum / Uhrzeit").asfreq("h")

        # set marker for the interpolation marker. True if respective df["Wert in mm"] is nan false if not
        was_nan = df["Wert in mm"].isna()

        # The interpolation of the NaNs in the column "Wert in mm" but just inside the data frame.
        df["Wert in mm"] = df["Wert in mm"].interpolate(method="time", limit=6, limit_direction="both", limit_area="inside")

        # new column which marks interpolated values
        df["Interpoliert"] = was_nan & df["Wert in mm"].notna()

       
        for c in ("Stations ID", "Geographische Breite", "Geographische Länge", "Stationsname"):
            if c in df.columns and df[c].notna().any():
                df[c] = df[c].fillna(df[c].iloc[0])
                
        # reset the index and change the "index" back to "Datum / Uhrzeit" to avoid mistakes for repeated execution. 
        df = df.reset_index().rename(columns={"index": "Datum / Uhrzeit"})

        # create new data frame with the respective columns
        cols = [c for c in ("Stations ID", "Stationsname", "Geographische Breite", "Geographische Länge", "Datum / Uhrzeit", "Wert in mm", "Interpoliert") if c in df.columns]

        # store those as a data frame
        df = df[cols]
        
        interpolated.append({
            "Datei": p.name, "Pfad": p,  "Data Frame": df
        })

    print(f"\n Ready. interpolated {len(interpolated)} of {len(files)} files.")
    return interpolated

def mask_big_gaps(interpolated):
    masked = []
    for i in interpolated:
        df_masked = i["Data Frame"].copy()
        filename = i["Datei"]
        path = i["Pfad"]

        df_masked["Maskiert"] = df_masked["Wert in mm"].isna()
        df_masked["Wert in mm"] = df_masked["Wert in mm"].fillna(-999.0)

        cols = [c for c in ("Stations ID", "Stationsname", "Geographische Breite", "Geographische Länge", "Datum / Uhrzeit", "Wert in mm", "Interpoliert", "Maskiert") if c in df_masked.columns]

        df_masked = df_masked[cols]

        # replace old ending with new ending and store it as a csv file
        out_path = path.with_name(filename.replace("_filtered_standardised", "_interpolated_and_masked"))
        df_masked.to_csv(out_path, index=False, encoding="utf-8")

        print(f"masked: {filename} to {out_path.name} ({len(df_masked)} rows)")
        masked.append({
            "Datei": out_path.name, "Pfad": out_path, "Data Frame": df_masked
        })

    print(f"\n Ready. masked {len(masked)} of {len(interpolated)} files.")
    return masked

        
    
def interpolate_and_mask():
    interpolated = interpolate_small_gaps()
    masked = mask_big_gaps(interpolated)

    return masked

def interpolate_and_big_gaps():
    interpolated = interpolate_small_gaps()
    gaps = []
    for i in interpolated:
        df_gaps = i["Data Frame"].copy()
        filename = i["Datei"]
        path = i["Pfad"]


        cols = [c for c in ("Stations ID", "Stationsname", "Geographische Breite", "Geographische Länge", "Datum / Uhrzeit", "Wert in mm", "Interpoliert") if c in df_gaps.columns]

        df_gaps = df_gaps[cols]

        # replace old ending with new ending and store it as a csv file
        out_path = path.with_name(filename.replace("_filtered_standardised", "_interpolated_and_nan"))
        df_gaps.to_csv(out_path, index=False, encoding="utf-8")

        print(f"just interpolated: {filename} to {out_path.name} ({len(df_gaps)} rows)")
        gaps.append({
            "Datei": out_path.name, "Pfad": out_path, "Data Frame": df_gaps
        })

    print(f"\n Ready. Interpolated small gaps an kept NaNs in big gaps in {len(gaps)} of {len(interpolated)} files.")
    return gaps

def keep_nan():
    kept = []

    for f in files:
        path = Path(f)
        filename = path.name

        df_nan = pd.read_csv(path, encoding="utf-8")

        # store the data frame as a copy of the data frame to work with it without missunderstandings.
        df_nan = df_nan.copy()
        # define the column "Datum / Uhrzeit" as a real date time.
        df_nan["Datum / Uhrzeit"] = pd.to_datetime(df_nan["Datum / Uhrzeit"], errors="coerce")

        # convert the column "Wert in mm to a real numeric value.
        df_nan["Wert in mm"] = df_nan["Wert in mm"].astype(float)

        # maybe "Datum / Uhrzeit" is by any case not sorted.
        df_nan = df_nan.sort_values("Datum / Uhrzeit")
        
        # set the column "Datum / Uhrzeit" as index and in hourly frequence. Now the gaps are rows with NaNs but the Index is a real date time.
        df_nan = df_nan.set_index("Datum / Uhrzeit").asfreq("h")


        
        for c in ("Stations ID", "Geographische Breite", "Geographische Länge", "Stationsname"):
            if c in df_nan.columns and df_nan[c].notna().any():
                df_nan[c] = df_nan[c].fillna(df_nan[c].iloc[0])
                
        # reset the index and change the "index" back to "Datum / Uhrzeit" to avoid mistakes for repeated execution. 
        df_nan = df_nan.reset_index().rename(columns={"index": "Datum / Uhrzeit"})

        cols = [c for c in ("Stations ID", "Stationsname", "Geographische Breite", "Geographische Länge", "Datum / Uhrzeit", "Wert in mm") if c in df_nan.columns]

        df_nan = df_nan[cols]

        # replace old ending with new ending and store it as a csv file
        out_path = path.with_name(filename.replace("_standardised", "_just_nan"))
        df_nan.to_csv(out_path, index=False, encoding="utf-8")

        print(f"just kept all NaNs: {filename} to {out_path.name} ({len(df_nan)} rows)")
        kept.append({
            "Datei": out_path.name, "Pfad": out_path, "Data Frame": df_nan
        })

    print(f"\n Ready. Kept NaNs in {len(kept)} of {len(files)} files.")
    return kept

def mask_nans():
    mask = []

    for f in files:
        path = Path(f)
        filename = path.name

        df_mask = pd.read_csv(path, encoding="utf-8")

        # store the data frame as a copy of the data frame to work with it without missunderstandings.
        df_mask = df_mask.copy()
        # define the column "Datum / Uhrzeit" as a real date time.
        df_mask["Datum / Uhrzeit"] = pd.to_datetime(df_mask["Datum / Uhrzeit"], errors="coerce")

        # convert the column "Wert in mms to a real numeric value.
        df_mask["Wert in mm"] = df_mask["Wert in mm"].astype(float)

        # maybe "Datum / Uhrzeit" is by any case not sorted.
        df_mask = df_mask.sort_values("Datum / Uhrzeit")
        
        # set the column "Datum / Uhrzeit" as index and in hourly frequence. Now the gaps are rows with NaNs but the Index is a real date time.
        df_mask = df_mask.set_index("Datum / Uhrzeit").asfreq("h")


        
        for c in ("Stations ID", "Geographische Breite", "Geographische Länge", "Stationsname"):
            if c in df_mask.columns and df_mask[c].notna().any():
                df_mask[c] = df_mask[c].fillna(df_mask[c].iloc[0])
                
        

        df_mask["Maskiert"] = df_mask["Wert in mm"].isna()
        df_mask["Wert in mm"] = df_mask["Wert in mm"].fillna(-999.0)

        # reset the index and change the "index" back to "Datum / Uhrzeit" to avoid mistakes for repeated execution. 
        df_mask = df_mask.reset_index().rename(columns={"index": "Datum / Uhrzeit"})
        
        cols = [c for c in ("Stations ID", "Stationsname", "Geographische Breite", "Geographische Länge", "Datum / Uhrzeit", "Wert in mm", "Maskiert") if c in df_mask.columns]

        df_mask = df_mask[cols]

        # replace old ending with new ending and store it as a csv file
        out_path = path.with_name(filename.replace("_standardised", "_just_masked"))
        df_mask.to_csv(out_path, index=False, encoding="utf-8")

        print(f"masked: {filename} to {out_path.name} ({len(df_mask)} rows)")
        mask.append({
            "Datei": out_path.name, "Pfad": out_path, "Data Frame": df_mask
        })

    print(f"\n Ready. masked {len(mask)} of {len(files)} files.")
    return mask

    

interpolated_and_masked = interpolate_and_mask()
interpolated_and_nan = interpolate_and_big_gaps()
kept_all_nans = keep_nan()
masked_nans = mask_nans()

Found 128 files

 Ready. interpolated 128 of 128 files.
masked: produkt_rr_stunde_20040814_20241231_00020_enhanced_filtered_standardised.csv to produkt_rr_stunde_20040814_20241231_00020_enhanced_interpolated_and_masked.csv (131496 rows)
masked: produkt_rr_stunde_20021101_20241231_00257_enhanced_filtered_standardised.csv to produkt_rr_stunde_20021101_20241231_00257_enhanced_interpolated_and_masked.csv (131496 rows)
masked: produkt_rr_stunde_20040701_20241231_00259_enhanced_filtered_standardised.csv to produkt_rr_stunde_20040701_20241231_00259_enhanced_interpolated_and_masked.csv (131496 rows)
masked: produkt_rr_stunde_20060524_20241231_00269_enhanced_filtered_standardised.csv to produkt_rr_stunde_20060524_20241231_00269_enhanced_interpolated_and_masked.csv (131496 rows)
masked: produkt_rr_stunde_20061101_20241231_00279_enhanced_filtered_standardised.csv to produkt_rr_stunde_20061101_20241231_00279_enhanced_interpolated_and_masked.csv (131496 rows)
masked: produkt_rr_stunde_20060724_2024